# Get TNS reports and classes and cross match with objects

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import random
import collections
import pickle
from collections import OrderedDict

In [2]:
import requests

URL = 'http://tns.snad.space/api/v1/circle'


def tns_cone_search(ra_deg, dec_deg, radius_arcsec):
    r = requests.get(
        URL,
        params=dict(ra=ra_deg, dec=dec_deg, radius_arcsec=radius_arcsec),
    )
    r.raise_for_status()
    return r.json()


class NotFound(RuntimeError):
    pass


class TwoOrMoreFound(RuntimeError):
    pass

def get_tns_query(ra_deg, dec_deg, radius_arcsec):
    objects = tns_cone_search(ra_deg, dec_deg, radius_arcsec)

    if len(objects) == 0:
        return 'No_TNS'
    if len(objects) > 1:
        print(TwoOrMoreFound(f'Too many objects for ra={ra_deg}, dec={dec_deg} found: {len(objects)}. Take TNS report reported first (not most recent).'))
        return objects[-1]

    obj = objects[0]
    #print(obj)
    return obj

In [3]:
#dataset_bank = pd.read_csv('../dataframes/dataset_bank_90k.csv', index_col='ztf_object_id')
dataset_bank = pd.read_csv(f'../loci_dbs/alerce_cut/merge_df_gr_alerce_sn_9420objects.csv.gz', compression='gzip', index_col='ztf_object_id')

dataset_bank = dataset_bank[['ra', 'dec']]
dataset_bank

/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (125,133) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ra,dec
ztf_object_id,,
ZTF17aaadars,74.008152,4.688007
ZTF17aaadvfh,10.123141,40.602475
ZTF17aaahrni,80.423285,-4.186247
ZTF17aaaivqs,127.370917,10.773342
ZTF17aaaizmo,132.155248,10.782338
...,...,...
ZTF22aacjbmo,205.509960,-19.933047
ZTF22aackync,302.025743,-2.811300
ZTF22aacreno,146.940490,-21.929953


In [4]:
get_tns_query(ra_deg=160.8872687, dec_deg=18.687709451612907, radius_arcsec=1.2)

{'objid': 76055,
 'name_prefix': 'SN',
 'name': '2021bfg',
 'ra': 160.887264134,
 'declination': 18.6877333276,
 'redshift': 0.058,
 'typeid': 105.0,
 'type': 'SN Iax[02cx-like]',
 'reporting_groupid': 4.0,
 'reporting_group': 'Pan-STARRS',
 'source_groupid': 4.0,
 'source_group': 'Pan-STARRS',
 'discoverydate': '2021-01-24 14:21:07.200',
 'discoverymag': 18.65,
 'discmagfilter': 26.0,
 'filter': 'w',
 'reporters': "K. C. Chambers, T. de Boer, J. Bulger, J. Fairlamb, M. Huber, C.-C. Lin, T. Lowe, E. Magnier, A. Schultz, R. J. Wainscoat, M. Willman (IfA, University of Hawaii), K. W. Smith, D. R. Young, O. McBrien, J. Gillanders, S. Srivastav, M. Fulton, S. J. Smartt, D. O'Neill, P. Clark, S. Sim (Queen's University Belfast), D. E. Wright (University of Minnesota)",
 'time_received': '2021-01-26 10:30:33',
 'internal_names': 'PS21tj, ZTF21aagqdvr, ATLAS21dpn',
 'creationdate': '2021-01-26 10:30:34',
 'lastmodified': '2021-02-27 01:06:49',
 'coord': {'ra': 160.88726413399982, 'dec': 18.68

In [5]:
get_tns_query(ra_deg=18.20966532571456, dec_deg=32.13815482993683, radius_arcsec=1.5)

Too many objects for ra=18.20966532571456, dec=32.13815482993683 found: 2. Take TNS report reported first (not most recent).


{'objid': 21527,
 'name_prefix': 'AT',
 'name': '2017fpd',
 'ra': 18.209667,
 'declination': 32.138089,
 'redshift': None,
 'typeid': None,
 'type': None,
 'reporting_groupid': 24.0,
 'reporting_group': 'GaiaAlerts',
 'source_groupid': 24.0,
 'source_group': 'GaiaAlerts',
 'discoverydate': '2017-07-19 05:00:57.000',
 'discoverymag': 15.75,
 'discmagfilter': 75.0,
 'filter': 'G',
 'reporters': 'A. Delgado, D. Harrison, S. Hodgkin, M. van Leeuwen, G. Rixon, A. Yoldas (University of Cambridge), on behalf of Gaia Alerts team',
 'time_received': '2017-07-22 19:23:02',
 'internal_names': 'Gaia17bum',
 'creationdate': '2017-07-22 19:23:03',
 'lastmodified': '2017-07-22 19:23:03',
 'coord': {'ra': 18.209666999999985, 'dec': 32.13808900000001}}

In [6]:
%%time

TNSresult_dict, noTNSresult_dict = {}, {}
group_name_full_dict, group_name_spec_dict, YSE_report_all_dict, YSE_report_class_dict = {}, {}, {}, {}

has_tns_l, ztf_ids_l, spec_cls_l, specz_l, report_group_l = [], [], [], [], []
for idx, (obj, ra, dec) in enumerate(zip(dataset_bank.index, dataset_bank['ra'], dataset_bank['dec'])):
    try: 
        tns_query = get_tns_query(ra_deg=ra, dec_deg=dec, radius_arcsec=1.5) # returns str or dict
    except: 
        print(f"tns_query for ra={ra}, dec={dec} has error. Skip!")
        continue
        
    ztf_ids_l.append(obj)
    
    if idx % 1000 == 0: print(idx)
    
    if isinstance(tns_query, str): 
        if (tns_query == 'No_TNS'): # Cone search is empty or has object w/ no TNS reporting (i.e star)
            noTNSresult_dict[obj] = tns_query
            has_tns_l.append('No_TNS')
            spec_cls_l.append('None'), specz_l.append(int(-99)), report_group_l.append('None')
            
        if (tns_query == 'TwoOrMoreFound'): # Cone search has two or more TNS reports at this ra/dec
            has_tns_l.append('Yes_TNS_2plus')
            spec_cls_l.append('None'), specz_l.append(int(-99)), report_group_l.append('None')
                
    else: # Cone search is NOT empty
        has_tns_l.append('Yes_TNS')
        #print(tns_query)
        tns_type = tns_query.get('type')
        #print(tns_query.get('creationdate'))
        redshift = tns_query.get('redshift')
        # Find reporting group
        report_group = tns_query.get('reporting_group')
        group_name_full_dict[obj] = report_group # all obj reporting
        
        
#         # of TNS queries, compile all which have YSE reporting group (even if aren't classified)
#         if tns_query.get('reporting_group') == 'YSE':
#             YSE_report_all_dict[obj] = tns_type if tns_type is not None else 'None'
    
        if tns_type is None: # Returns 404 cone search. 
            noTNSresult_dict[obj] = 'None'
            spec_cls_l.append('None')
            specz_l.append(int(-99))
            report_group_l.append(report_group)

        #elif tns_type is ma.masked: # Has cone search, returned obj is awaiting TNS classification
        #    TNSresult_dict[obj] = 'awaiting_class'

        else: # TNS has a classification, assign it
            TNSresult_dict[obj] = tns_type 
            spec_cls_l.append(tns_type), specz_l.append(redshift), report_group_l.append(report_group)

            # Find reporting group of spec obj
            group_name_spec_dict[obj] = report_group
            
#             # Get subset of objects reported by YSE
#             if report_group == 'YSE':
#                 YSE_report_class_dict[obj] = tns_type

0
Too many objects for ra=185.1877845394248, dec=49.55128449550819 found: 2. Take TNS report reported first (not most recent).
Too many objects for ra=246.6787965505368, dec=13.113863399756426 found: 2. Take TNS report reported first (not most recent).
1000
Too many objects for ra=17.7471574, dec=0.4336238998396311 found: 2. Take TNS report reported first (not most recent).
2000
Too many objects for ra=39.6317390503808, dec=-8.262769724909404 found: 2. Take TNS report reported first (not most recent).
Too many objects for ra=249.66427172614863, dec=33.678278841064134 found: 2. Take TNS report reported first (not most recent).
Too many objects for ra=15.5282847, dec=1.2757815951419644 found: 2. Take TNS report reported first (not most recent).
Too many objects for ra=5.597013311390017, dec=40.003946886436765 found: 2. Take TNS report reported first (not most recent).
Too many objects for ra=139.10614049178756, dec=47.24487500851932 found: 2. Take TNS report reported first (not most rece

In [7]:
dataset_bank_tns_df = pd.DataFrame(zip(ztf_ids_l, has_tns_l, spec_cls_l, specz_l, report_group_l), columns=['ztf_object_id', 'has_tns', 'tns_cls', 'spec_z', 'report_group'])
dataset_bank_tns_df = dataset_bank_tns_df.set_index('ztf_object_id')
dataset_bank_tns_df.to_csv(f'../loci_dbs/alerce_cut/merge_df_gr_alerce_sn_{len(dataset_bank_tns_df)}objects_tns_df.csv.gz', compression='gzip')
dataset_bank_tns_df

,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,
ZTF17aaadars,No_TNS,None,-99.0,None
ZTF17aaadvfh,No_TNS,None,-99.0,None
ZTF17aaahrni,Yes_TNS,None,-99.0,GaiaAlerts
ZTF17aaaivqs,No_TNS,None,-99.0,None
ZTF17aaaizmo,No_TNS,None,-99.0,None
...,...,...,...,...
ZTF22aacjbmo,No_TNS,None,-99.0,None
ZTF22aackync,No_TNS,None,-99.0,None
ZTF22aacreno,No_TNS,None,-99.0,None


In [8]:
# Save all dicts as pickle

list_of_dicts = [TNSresult_dict, noTNSresult_dict, group_name_full_dict, group_name_spec_dict]
list_of_dicts_str = ['TNSresult_dict_9k', 'noTNSresult_dict_9k', 'group_name_full_dict_9k', 'group_name_spec_dict_9k']

for dic, dic_str in zip(list_of_dicts, list_of_dicts_str):
    with open(f'../loci_dbs/alerce_cut/{dic_str}.pickle', 'wb') as handle:
        pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
from collections import Counter
Counter(dataset_bank_tns_df.tns_cls)

Counter({'None': 6732,
         'SN Ia': 1680,
         'TDE': 24,
         'SN Ic-BL': 25,
         'SN II': 425,
         'SN Ia-pec': 18,
         'SN Ib': 39,
         'SN IIn': 96,
         'SLSN-I': 35,
         'SN Ia-91T-like': 64,
         'SN IIP': 39,
         'CV': 46,
         'SN Ic': 38,
         'SN': 8,
         'SLSN-II': 24,
         'Varstar': 10,
         'SN Ia-CSM': 5,
         'AGN': 10,
         'SN IIb': 34,
         'SN Ic-Ca-rich': 1,
         'LRN': 1,
         'SN IIL': 1,
         'SN Ibn': 9,
         'LBV': 2,
         'SN Iax[02cx-like]': 5,
         'Nova': 6,
         'SN Ia-91bg-like': 9,
         'SN I': 8,
         'SN Ib-pec': 3,
         'SN II-pec': 2,
         'SN Ib/c': 8,
         'Other': 7,
         'SN Icn': 1,
         'SN IIn-pec': 1,
         'SN Ia-SC': 2,
         'Galaxy': 1,
         'TDE-H-He': 1})

In [10]:
Counter(dataset_bank_tns_df.has_tns)

Counter({'No_TNS': 2080, 'Yes_TNS': 7340})

In [13]:
dataset_bank = pd.read_csv(f'../loci_dbs/alerce_cut/merge_df_gr_alerce_sn_9420objects.csv.gz', compression='gzip', index_col='ztf_object_id')

dataset_bank_and_tns_df = pd.merge(dataset_bank, dataset_bank_tns_df, left_index=True, right_index=True)
dataset_bank_and_tns_df.to_csv(f'../loci_dbs/alerce_cut/dataset_bank_and_tns_df_9420objects.csv.gz', compression='gzip')
dataset_bank_and_tns_df

/Users/patrickaleo/miniconda3/envs/laiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (125,133) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,locus_id,ra,dec,tags,catalogs,ztf_ssnamenr,num_alerts,num_mag_values,oldest_alert_id,...,vpdf_extreme_version,vpdf_extreme_faint,vpdf_extreme_bright,locus_gal_l,locus_gal_b,stamp_cls,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaadars,858089,ANT2020setxa,74.008152,4.688007,"['dwarf_nova_outburst', 'high_snr', 'high_ampl...",['bright_guide_star_cat'],NaN,176,50,ztf_candidate:726272534215015034,...,NaN,NaN,NaN,194.434930,-23.001128,SNIa,No_TNS,None,-99.0,None
ZTF17aaadvfh,845168,ANT2020jk2po,10.123141,40.602475,"['in_m31', 'lc_feature_extractor', 'extragalac...","['RC3', '2mass_xsc', 'gaia_dr2']",NaN,345,25,ztf_candidate:1007342854215010039,...,NaN,NaN,NaN,120.687977,-22.221579,SLSN,No_TNS,None,-99.0,None
ZTF17aaahrni,896108,ANT2020aevgf7q,80.423285,-4.186247,"['high_amplitude_transient_candidate', 'nova_t...",[],NaN,63,16,ztf_candidate:1398444022315015011,...,NaN,NaN,NaN,206.213250,-21.841968,SNIa,Yes_TNS,None,-99.0,GaiaAlerts
ZTF17aaaivqs,985055,ANT2020jq7w6,127.370917,10.773342,"['high_amplitude_variable_star_candidate', 'lc...","['asassn_variable_catalog_v2_20190802', 'allwi...",NaN,358,292,ztf_candidate:671461312615010000,...,NaN,NaN,NaN,214.180505,26.612885,SLSN,No_TNS,None,-99.0,None
ZTF17aaaizmo,998136,ANT2020pg5q4,132.155248,10.782338,"['matheson_extreme_vpdf', 'lc_feature_extracto...","['2mass_psc', 'bright_guide_star_cat', 'gaia_d...",NaN,289,97,ztf_candidate:680499561915015002,...,0.1.0,pct_99.99999_outlier,NaN,216.450964,30.870773,SNII,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aacjbmo,909153,ANT2020euljm,205.509960,-19.933047,['lc_feature_extractor'],"['allwise', 'bright_guide_star_cat', '2mass_ps...",NaN,132,60,ztf_candidate:738499311115015008,...,NaN,NaN,NaN,318.854533,41.368904,SLSN,No_TNS,None,-99.0,None
ZTF22aackync,274345,ANT2020aeh34ei,302.025743,-2.811300,['lc_feature_extractor'],"['bright_guide_star_cat', '2mass_psc', 'allwis...",NaN,273,88,ztf_candidate:1357212311615015001,...,NaN,NaN,NaN,39.334932,-18.312848,SNII,No_TNS,None,-99.0,None
ZTF22aacreno,281787,ANT2020edvwq,146.940490,-21.929953,['lc_feature_extractor'],"['milliquas', 'bright_guide_star_cat', 'gaia_d...",NaN,129,95,ztf_candidate:828188395815015007,...,NaN,NaN,NaN,256.172074,23.831250,SLSN,No_TNS,None,-99.0,None


In [14]:
dataset_bank_and_tns_df.loc['ZTF22aaezyos']

KeyError: 'ZTF22aaezyos'